In [2]:
import os
import re
import csv
import pickle
import googleapiclient.errors
import google_auth_oauthlib.flow
import googleapiclient.discovery

api_version = "v3"
api_service_name = "youtube"
client_secrets_file = "client_secrets.json"
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
filters = ['vk','music','instagram','tiktok']

In [3]:
def get_auth_service():
    cpf = "DELETE_FILE"
    # cpf = "CREDENTIALS_PICKLE_FILE"
    if os.path.exists(cpf):
        with open(cpf, 'rb') as f:
            credentials = pickle.load(f)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes, redirect_uri="https://localhost:6969/")
        auth_url = flow.authorization_url()
        print("URL IS: ", auth_url)
        credentials = flow.run_console()
        # credentials = flow.run_local_server(host='localhost',
        #                                     port=6969, 
        #                                     authorization_prompt_message='Please visit this URL: {url}', 
        #                                     success_message='The auth flow is complete; you may close this window.',
        #                                     open_browser=True)
        with open(cpf, 'wb') as f:
            pickle.dump(credentials, f)
    return googleapiclient.discovery.build(api_service_name, api_version, credentials = credentials)

# def get_auth_service():
#     cpf = "CREDENTIAL_PICKLE_FILE"
#     # cpf = "CREDENTIALS_PICKLE_FILE"
#     if os.path.exists(cpf):
#         with open(cpf, 'rb') as f:
#             credentials = pickle.load(f)
#     else:
#         flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
#         auth_url = google_auth_oauthlib.flow.authorization_url
#         credentials = flow.run_console()
#         with open(cpf, 'wb') as f:
#             pickle.dump(credentials, f)
#     return googleapiclient.discovery.build(api_service_name, api_version, credentials = credentials)

In [4]:
def get_latest_video_description(channel_id):
    youtube = get_auth_service()

    response = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    ).execute()

    # get "uploads" playlist ID
    channel = response['items'][0]
    contentDetails = channel['contentDetails']
    relatedPlaylists = contentDetails['relatedPlaylists']
    uploads = relatedPlaylists['uploads']
    
    playlist = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=1,
        playlistId=uploads
    ).execute()

    videos = playlist['items']
    latest_video = videos[0]
    latest_video_description = latest_video['snippet']['description']
    return latest_video_description



get_latest_video_description("UC-lHJZR3Gqxm24_Vd_AJ5Yw")

URL IS:  ('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Flocalhost%3A6969%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=AaZoejiITWsMumTBwFjZwVXQzJEAxX&access_type=offline', 'AaZoejiITWsMumTBwFjZwVXQzJEAxX')
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=yP512L5f3GWqkiTdGGRHXop4Qtc5FY&prompt=consent&access_type=offline


ValueError: Please supply either code or authorization_response parameters.

In [ ]:
def get_links(description):
    return re.findall("(?P<url>https?://[^\s]+)", description)

In [ ]:
def filter_links(links):
    return [link for link in links if not re.search("|".join(["("+f+")" for f in filters]),link)]

In [ ]:
def parse():
    opened_file = open('input.csv', encoding="UTF-8")
    reader = csv.reader(opened_file)

    for row in reader: 
        try:
            channel_id = row[2]
            description = get_latest_video_description(channel_id)
            links = get_links(description)
            filtered_links = filter_links(links)
            print(filtered_links)
        except:
            print(reader.line_num)

In [ ]:
# this method allows developer to specify the number of videos to get links from
def get_links_from_videos(channel_id,number_of_videos=1):
    youtube = get_auth_service()

    response = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    ).execute()

    # get "uploads" playlist ID
    channel = response['items'][0]
    contentDetails = channel['contentDetails']
    relatedPlaylists = contentDetails['relatedPlaylists']
    uploads = relatedPlaylists['uploads']
    
    playlist = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=number_of_videos,
        playlistId=uploads
    ).execute()

    videos = playlist['items']
    all_links = []
    for i in range(number_of_videos):
        video_description = videos[i]['snippet']['description']
        links = get_links(video_description)
        filtered_links = filter_links(links)
        all_links += filtered_links
    #here I return unique links
    return list(dict.fromkeys(all_links))

In [ ]:
pewds = 'UC-lHJZR3Gqxm24_Vd_AJ5Yw'
links = get_links_from_videos(pewds,10)

print(links)

['https://www.youtube.com/channel/UC-lHJZR3Gqxm24_Vd_AJ5Yw/join', 'https://gfuel.ly/31Kargr', 'https://represent.com/store/pewdiepie', 'https://tsuki.market/', 'https://www.based.gg', 'https://pewdiepie.store/', 'https://rhinoshield.io/pewdiepie', 'https://terraria.shop/collections/pewdiepie', 'https://clutchchairz.com/pewdiepie/', 'https://ghostkeyboards.com/pages/pewdiepie', 'https://buff.ly/2pNG0aT', 'https://buff.ly/34C68nZ', 'https://apps.apple.com/us/app/pewdiepies-tuber-simulator/id1093190533', 'https://play.google.com/store/apps/details?id=com.outerminds.tubular&hl=en_GB&gl=US', 'https://youtu.be/FQgLsYOKP8w', 'https://NordVPN.com/pewdiepie', 'https://www.youtube.com/watch?v=CD7UmmLhFR8', 'https://store.steampowered.com/app/979070/Just_Die_Already/', 'https://www.youtube.com/channel/UCepq9z9ovYGxhNrvf6VMSjg', 'https://gfuel.ly/pewdiepie']


In [ ]:
def perform(func):
    def inner():
        x = 6
        y = 9
        func(x,y)
    return inner

@perform
def add(x,y):
    z = x+y
    print(z)

@perform 
def mult(x,y):
    z = x*y
    print(z)

add()
mult()

15
54


In [7]:
# returns link for user to follow
def get_auth_service_link():
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes, redirect_uri="urn:ietf:wg:oauth:2.0:oob")
    url = flow.authorization_url()
    print(url)
get_auth_service_link()

('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=aXhq5XJvtdqtDRlR8lXbz10jcz84OV&access_type=offline', 'aXhq5XJvtdqtDRlR8lXbz10jcz84OV')


In [11]:
# returns link for user to follow
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes, redirect_uri="urn:ietf:wg:oauth:2.0:oob")

def get_auth_service_link():
    url = flow.authorization_url()
    print(url)
get_auth_service_link()

def authorize(code):
    flow.fetch_token(code=code)
    credentials = flow.credentials
    print("success!")

('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=MztNP4WqOTpFnEDqgVIkhCA3z6icAM&access_type=offline', 'MztNP4WqOTpFnEDqgVIkhCA3z6icAM')


In [12]:
authorize("4/1AX4XfWjB2uk907uckUoLwF_6dcoevBCQMGo2WekLq8d5FykTITFtI5hhdtk")

success!


In [3]:
def m1():
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes, redirect_uri="urn:ietf:wg:oauth:2.0:oob")
    url = flow.authorization_url()
    print(url)

def m2(code):
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes, redirect_uri="urn:ietf:wg:oauth:2.0:oob")
    flow.fetch_token(code=code)
    credentials = flow.credentials
    print("success!")

m1()

('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=DVqEzS8SBqdguLIKALFEnty1UrsKdY&access_type=offline', 'DVqEzS8SBqdguLIKALFEnty1UrsKdY')


In [4]:
m2("4/1AX4XfWh-BKImKTzBZozgssp75Cnjv1Y8WRDb0I60Mb-Y9gyS6Mshd6MU2zQ")

success!
